In [1]:
import tensorflow as tf
import pandas as pd
from PIL import Image
import io
import numpy as np
import random

# Image Stream

In [2]:
# image file path
# bbox
# true label

In [3]:
train1= pd.read_csv("body_posture_dataset/train_0.csv")
train2= pd.read_csv("body_posture_dataset/train_2.csv")
train3= pd.read_csv("body_posture_dataset/train_1.csv")

In [4]:
filename1 = train1["filename"].tolist()
xmin1 = train1["xmin"].tolist()
ymin1 = train1["ymin"].tolist()
xmax1 = train1["xmax"].tolist()
ymax1 = train1["ymax"].tolist()

In [5]:
filename2 = train2["filename"].tolist()
xmin2 = train2["xmin"].tolist()
ymin2 = train2["ymin"].tolist()
xmax2 = train2["xmax"].tolist()
ymax2 = train2["ymax"].tolist()

In [6]:
filename3 = train3["filename"].tolist()
xmin3 = train3["xmin"].tolist()
ymin3 = train3["ymin"].tolist()
xmax3 = train3["xmax"].tolist()
ymax3 = train3["ymax"].tolist()

In [7]:
stream = []
for i, _ in enumerate(filename1):
    temp = [filename1[i], [xmin1[i], ymin1[i], xmax1[i], ymax1[i]], 0] 
    stream.append(temp)

In [8]:
for i, _ in enumerate(filename2):
    temp = [filename2[i], [xmin2[i], ymin2[i], xmax2[i], ymax2[i]], 2] 
    stream.append(temp)

In [9]:
for i, _ in enumerate(filename3):
    temp = [filename3[i], [xmin3[i], ymin3[i], xmax3[i], ymax3[i]], 1] 
    stream.append(temp)

In [10]:
#stream = np.asarray(stream)
#random.seed(0)
random.shuffle(stream)

In [11]:
pd.DataFrame(stream)

,0,1,2
0,aug_2_1_0_9502.jpg,"[104.0, 106.0, 152.0, 165.0]",2
1,200787.jpg,"[92.0, 164.0, 139.0, 241.0]",1
2,200953.jpg,"[70.0, 94.0, 131.0, 197.0]",0
3,aug_2_1_0_6002.jpg,"[53.0, 162.0, 101.0, 218.0]",2
4,aug_1_1_0_2838.jpg,"[271.0, 147.0, 335.0, 247.0]",0
...,...,...,...
13495,300539.jpg,"[96.0, 149.0, 149.0, 189.0]",1
13496,aug_1_3_0_6782.jpg,"[198.0, 86.0, 261.0, 183.0]",0
13497,100198.jpg,"[92.0, 154.0, 149.0, 221.0]",1
13498,aug_1_3_0_6684.jpg,"[265.0, 84.0, 355.0, 179.0]",0


# Inference

In [12]:
class Model:
    def __init__(self):
        #MODEL_PATH='model/basic-cnn-functional.h5'
        #MODEL_PATH='model/basic-cnn-functional-expert.h5'
        #MODEL_PATH='model/basic-cnn-sequential.h5'
        self.MODEL_PATH='model/basic-cnn-functional.h5'
        self.model = tf.keras.models.load_model(self.MODEL_PATH)
        # inference를 할 경우엔 warning 무시
        # training을 할 경우엔 compile 해야함
    
    def inference(self, image):
        prediction = self.model(image)
        #self.model.predict(image)
        return np.argmax(prediction)
    
    def get_filepath(self, filename, label):
        return "body_posture_dataset/"+str(label)+"/"+filename
    
    def preprocess_image(self, image, bbox):
        img = Image.open(image).convert("L") # convert to grayscale
        img = img.crop(bbox)
        img = img.resize((50,50))
        img = np.asarray(img)/255.0
        img = np.expand_dims(img, -1)
        img = np.expand_dims(img, 0)
        return img

In [13]:
# 모듈로 만들어서 import해서 사용하기
class FallDetection:
    def __init__(self, buffer_size, fall_threshold, long_lie_threshold):
        self.buffer_size = buffer_size
        self.fall_threshold = fall_threshold
        self.long_lie_threshold = long_lie_threshold
        
        self.buffer = []
        self.long_lie_window = []
        self.lying_cnt = 0
        
        
        self.STANDING = 0
        self.LYING = 1
        self.BENDING = 2
        
       

    
    def buffer_step(self, label):
        #self.lying_cnt = 0
        self.buffer.append(label)
        
        if len(self.buffer)>self.buffer_size:
            self.buffer.pop(0)
            

    def detect_fall(self):
        # st: standing timestamp
        # lt: lying timestamp
        for st, label in enumerate(self.buffer):
            if label==self.STANDING:
                for lt in range(st, st+self.fall_threshold):
                    if lt>len(self.buffer)-1:
                        break
                    if self.buffer[lt] == self.LYING:
                        self.st = st
                        self.lt = lt
                        return True
        
        return False
    
    
    def detect_long_lie(self):
        
        self.lying_cnt = 0
        self.long_lie_window = []
        
        #1. declaring sliding window
        for t in range(self.lt, self.lt+self.long_lie_threshold):
            if t>len(self.buffer)-1:
                break
            
            self.long_lie_window.append(self.buffer[t])
            
            # initiate lying count
            if self.buffer[t] == self.LYING:
                self.lying_cnt +=1
        
        # alarm condition
        if self.lying_cnt >= self.long_lie_threshold:
            return True
        
        else:
            return False
        

    
    def generate_alarm(self):
        print("[ALERT] fall-down has just occurred!")
        print("fall detected between "+ str(self.st) + " and "+ str(self.lt) )
        print("self.buffer: ", self.buffer)
        print("--------")
        
        

In [14]:
model = Model()
fd = FallDetection(20, 5, 7) #buffer size, fall_threshold, long_lie_threshold

In [15]:
predictions = []
for item in stream[:3000]:
    image = item[0]
    bbox = item[1]
    label = item[2]
    image = model.get_filepath(image, label)
    image = model.preprocess_image(image, bbox)
    
    # inference
    prediction = model.inference(image)
    predictions.append(prediction)
    
    # fall detection
    fd.buffer_step(prediction)
    if fd.detect_fall() and fd.detect_long_lie():
        fd.generate_alarm()
        

[ALERT] fall-down has just occurred!
fall detected between 2 and 6
self.buffer:  [1, 1, 0, 2, 0, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 2]
--------
[ALERT] fall-down has just occurred!
fall detected between 1 and 5
self.buffer:  [1, 0, 2, 0, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 2, 1]
--------
[ALERT] fall-down has just occurred!
fall detected between 0 and 4
self.buffer:  [0, 2, 0, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 2, 1, 1]
--------
[ALERT] fall-down has just occurred!
fall detected between 1 and 3
self.buffer:  [2, 0, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 2, 1, 1, 2]
--------
[ALERT] fall-down has just occurred!
fall detected between 0 and 2
self.buffer:  [0, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 2, 1, 1, 2, 2]
--------
[ALERT] fall-down has just occurred!
fall detected between 2 and 3
self.buffer:  [2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 0, 2, 1]
--------
[ALERT] fall-down has just occurred!
fall detected between 1 and 2
self.buffer:  [1, 0, 1, 1, 

In [16]:
len(predictions)

3000

In [17]:
stream = np.asarray(stream)

In [18]:
predictions = np.asarray(predictions)

In [19]:
true_labels = np.asarray(stream[:, 2])

In [20]:
true_labels_clip = true_labels[:1000]

In [21]:
count = 0
for idx, item in enumerate(true_labels_clip):
    if true_labels_clip[idx] == predictions[idx]:
        count+=1
acc = count / len(predictions)

In [22]:
acc

0.3313333333333333